# 多进程并行 (MPI)

在终端使用 `mpiexec -n <np>` 运行 python 文件即可:
    
```bash
mpiexec -n 2 python myscript.py
```


<!--
```bash
MPICH_NO_LOCAL=1 mpiexec -n 16 -bind-to core -map-by socket python /path/to/script.py
```
-->

## Run code parallelly in jupyter

我们使用 `ipyparallel` 介绍并行程序的一些内容, 需要先安装 `ipyparallel`

### Install ipyparallel
1. Install ipyparallel in firedrake env:
    
    ```bash
    pip install ipyparallel
    ```

2. create profile mpi
    
    ```bash
    ipython profile create --parallel --profile=mpi
    ```
    
    Your will see the following output
    
    ```bash
    [ProfileCreate] Generating default config file: 
        PosixPath('/home/<your-user-name>/.ipython/profile_mpi/ipython_config.py')
    [ProfileCreate] Generating default config file: 
        PosixPath('/home/<your-user-name>/.ipython/profile_mpi/ipython_kernel_config.py')
    [ProfileCreate] Generating default config file: 
        PosixPath('/home/<your-user-name>/.ipython/profile_mpi/ipcontroller_config.py')
    [ProfileCreate] Generating default config file: 
        PosixPath('/home/<your-user-name>/.ipython/profile_mpi/ipengine_config.py')
    [ProfileCreate] Generating default config file: 
        PosixPath('/home/<your-user-name>/.ipython/profile_mpi/ipcluster_config.py')
    ```

3. Edit file `.ipython/profile_mpi/ipengine_config.py`. Add the following code at the begining of the file:
    
    ```python
    from firedrake import *
    from firedrake.petsc import PETSc
    ```

4. Set the default engines to mpi in file `.ipython/profile_mpi/ipcluster_config.py`. 
   You can search `engine_launcher_class` in the file, and the result file should looks like this:
    
    ```python
    #    - sshproxy: ipyparallel.cluster.launcher.SSHProxyEngineSetLauncher
    #    - winhpc: ipyparallel.cluster.launcher.WindowsHPCEngineSetLauncher
    #  Default: 'ipyparallel.cluster.launcher.LocalEngineSetLauncher'
    c.Cluster.engine_launcher_class = 'mpi'
    ```

4. Test:
    
    ```python
    import ipyparallel as ipp
    import os

    cluster = ipp.Cluster(profile="mpi", n=2)
    client = cluster.start_and_connect_sync()
    ```
    
    The output should looks like
    
    ```bash
    Starting 2 engines with <class 'ipyparallel.cluster.launcher.MPIEngineSetLauncher'>
    ```
    
    ```python
    %%px --block
    from firedrake import *
    from firedrake.petsc import PETSc
    from mpi4py import MPI

    mesh = RectangleMesh(8, 8, 1, 1)
    PETSc.Sys.syncPrint(mesh.comm.rank, mesh.comm.size)
    PETSc.Sys.syncFlush()
    ```
    
    The output should looks like:
    
    ```bash
    [stdout:0] 0 2
    1 2
    ```

### Example

In [ ]:
import ipyparallel as ipp
import os

cluster = ipp.Cluster(profile="mpi", n=2)
client = cluster.start_and_connect_sync()

Starting 2 engines with <class 'ipyparallel.cluster.launcher.MPIEngineSetLauncher'>


  0%|          | 0/2 [00:00<?, ?engine/s]

In [ ]:
%%px --block
from firedrake import *
from firedrake.petsc import PETSc
from mpi4py import MPI

mesh = RectangleMesh(8, 8, 1, 1)
PETSc.Sys.syncPrint(mesh.comm.rank, mesh.comm.size)
PETSc.Sys.syncFlush()

[stdout:0] 0 2
1 2


In [ ]:
%%px --block

PETSc.Sys.syncPrint(COMM_WORLD.rank, COMM_WORLD.size)
PETSc.Sys.syncFlush()

[stdout:0] 0 2
1 2


有些时候需要在某个进程上, 做指定的操作或运算, 如只在第0个进程上画图

```python
if COMM_WORLD.rank == 0:
    plot(...)
```

## 并行输出

[py/intro_utils.py](py/intro_utils.py)

In [ ]:
%%px --block 
from firedrake import *
from firedrake.petsc import PETSc
from mpi4py import MPI

PETSc.Sys.Print('This is first line (from rank 0)')

[stdout:0] This is first line (from rank 0)


In [ ]:
%%px --block 
PETSc.Sys.syncPrint('This is second line (from all rank)')
PETSc.Sys.syncFlush()

[stdout:0] This is second line (from all rank)
This is second line (from all rank)


In [ ]:
%%px --block
print('This msg from all rank')

[stdout:0] This msg from all rank


[stdout:1] This msg from all rank
